In [0]:
/* ADS_STG_SMA_CASE_PHASE_PROPERTIES_UPDATE */ 
TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_UPDATE;


INSERT INTO gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_UPDATE 
      select 
      			CPPROP_KEY
      			, CPPROP_SOURCE_ID
      			, CPPROP_SOURCE_SYSTEM_ID
      			, CPPROP_SOURCE_SYS_ORIGIN
      			, CPPROP_PARENT_SOURCE_ID
      			, CPPROP_PARENT_KEY
      			, CPPTP_KEY
      			, DUPL_RNK
      	from 
      	( 
WITH cpp_types AS (
    SELECT 
        t.cpptp_source_id,
        t.cpptp_key,
        SUBSTR(t.cpptp_source_id, INSTR(t.cpptp_source_id, '|') + 1) AS ct,
        LENGTH(t.CPPTP_PATH_LIKE) AS mask_length,
        SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1) AS cpptp_source_id_99,
        SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1) || '%' AS cpptp_source_id_44,
        CASE -- optimalize pro nahradu liku uvnitr retezce
            WHEN -- pokud je % ve stringu a je na konci retezce tak nic
                INSTR(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1), '%') > 0 
                AND INSTR(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1), '%') = LENGTH(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1))
            THEN SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1)
            WHEN -- pokud je % ve stringu a neni na konci retezce tak pridam na konec
                INSTR(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1), '%') > 0 
                AND INSTR(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1), '%') <> LENGTH(SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1))
            THEN SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1) || '%'
            ELSE SUBSTR(t.cpptp_source_id, 1, INSTR(t.cpptp_source_id, '|') - 1) || '%' -- kdyz nesplnuje ani jednu podminku tak pridej % na konec
        END AS cpptp_source_id2
    FROM gap_catalog.ads_src_20250901.case_phase_property_types t
    WHERE t.cpptp_source_system_id = 'SMA'
        AND t.cpptp_source_sys_origin = 'SMA_MONITOR_EVENTS'
        AND t.cpptp_valid_to = to_date('30000101','yyyyMMdd')
)
, src as (
select
        src.cpprop_key                  as cpprop_key,
        src.cpprop_source_id        ,
        src.cpprop_source_system_id ,
        src.cpprop_source_sys_origin,
        src.cpprop_parent_source_id     as cpprop_parent_source_id,
        nvl(par.cpprop_key,-1)          as cpprop_parent_key,
        src.attr_fp,
        src.casetype,
        src.cpptp_source_id,
        par.cpprop_key                  as par_cpprop_key
from    gap_catalog.ads_owner.sma_case_phase_properties src
left join gap_catalog.ads_owner.sma_case_phase_properties par 
on      par.cpprop_source_id = src.cpprop_parent_source_id
)  --select count(*) from src -- 8,418,942 rows 5 sec
, gr as ( 
select 
        src_gr.*,
        cpp_types.cpptp_key,
        cpp_types.mask_length
from    (   
            select
                    substr(src.cpptp_source_id, 1, 1)   as cpptp_source_id_1,
                    upper(src.casetype)                 as casetype,
                    src.attr_fp
            from    src        
            group by
                    substr(src.cpptp_source_id, 1, 1)   ,
                    upper(src.casetype)                 ,
                    src.attr_fp
         ) src_gr      -- 252,877        
left join cpp_types 
on  src_gr.cpptp_source_id_1    = substr(cpp_types.cpptp_source_id, 1, 1)
and src_gr.casetype             =  cpp_types.ct
and src_gr.attr_fp              like cpp_types.cpptp_source_id_99
)
select 
        x.cpprop_key,
        x.cpprop_source_id,
        x.cpprop_source_system_id,
        x.cpprop_source_sys_origin,
        x.cpprop_parent_source_id,
        x.cpprop_parent_key,
        x.cpptp_key,
        x.dupl_rnk
from (
        select
                src.cpprop_key                  as cpprop_key,
                src.cpprop_source_id        ,
                src.cpprop_source_system_id ,
                src.cpprop_source_sys_origin,
                src.cpprop_parent_source_id     as cpprop_parent_source_id,
                src.cpprop_parent_key,
                src.attr_fp,
                src.casetype,
                src.cpptp_source_id,
                src.par_cpprop_key,
                nvl(gr.cpptp_key,-1)     as cpptp_key,
                case
                    when    nvl(src.par_cpprop_key,-1)      > -1 
                    or      nvl(gr.cpptp_key,-1) > -1
                        then 1
                        else 0
                end                             as final_filter,
                row_number() over (partition by src.cpprop_key 
                                    order by gr.mask_length desc) as dupl_rnk
        from    src
        left join gr
        on      substr(src.cpptp_source_id, 1, 1)   = gr.cpptp_source_id_1
        and     upper(src.casetype)                 = gr.casetype
        and     src.attr_fp                         = gr.attr_fp
        ) x
where   x.dupl_rnk      = 1 
and     x.final_filter  = 1
)
;


/*MERGE*/
MERGE INTO gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES trg
USING
(
    SELECT CPPROP_KEY, 
           CPPROP_SOURCE_ID, 
           CPPROP_SOURCE_SYSTEM_ID, 
           CPPROP_SOURCE_SYS_ORIGIN, 
           CPPROP_PARENT_SOURCE_ID, 
           CPPROP_PARENT_KEY, 
           CPPTP_KEY, 
           DUPL_RNK
    FROM gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_UPDATE xc
) src
ON (trg.CPPROP_KEY = src.CPPROP_KEY
    AND trg.CPPROP_SOURCE_ID = src.CPPROP_SOURCE_ID
    AND trg.CPPROP_SOURCE_SYSTEM_ID = src.CPPROP_SOURCE_SYSTEM_ID
    AND trg.CPPROP_SOURCE_SYS_ORIGIN = src.CPPROP_SOURCE_SYS_ORIGIN)
WHEN MATCHED AND (
    NOT (src.CPPROP_PARENT_SOURCE_ID <=> trg.CPPROP_PARENT_SOURCE_ID) OR
    NOT (src.CPPROP_PARENT_KEY <=> trg.CPPROP_PARENT_KEY) OR
    NOT (src.CPPTP_KEY <=> trg.CPPTP_KEY) OR
    NOT (src.DUPL_RNK <=> trg.DUPL_RNK)
) THEN UPDATE SET 
    trg.CPPROP_PARENT_SOURCE_ID = src.CPPROP_PARENT_SOURCE_ID, 
    trg.CPPROP_PARENT_KEY = src.CPPROP_PARENT_KEY, 
    trg.CPPTP_KEY = src.CPPTP_KEY, 
    trg.DUPL_RNK = src.DUPL_RNK, 
    trg.SMCPP_UPDATED_DATETIME = CURRENT_TIMESTAMP(), 
    trg.SMCPP_UPDATE_PROCESS_KEY = 13165092
WHEN NOT MATCHED THEN INSERT 
(
    CPPROP_KEY, 
    CPPROP_SOURCE_ID, 
    CPPROP_SOURCE_SYSTEM_ID, 
    CPPROP_SOURCE_SYS_ORIGIN, 
    CPPROP_PARENT_SOURCE_ID, 
    CPPROP_PARENT_KEY, 
    CPPTP_KEY, 
    DUPL_RNK, 
    SMCPP_INSERTED_DATETIME, 
    SMCPP_INSERT_PROCESS_KEY, 
    SMCPP_UPDATED_DATETIME, 
    SMCPP_UPDATE_PROCESS_KEY
) 
VALUES 
(
    src.CPPROP_KEY, 
    src.CPPROP_SOURCE_ID, 
    src.CPPROP_SOURCE_SYSTEM_ID, 
    src.CPPROP_SOURCE_SYS_ORIGIN, 
    src.CPPROP_PARENT_SOURCE_ID, 
    src.CPPROP_PARENT_KEY, 
    src.CPPTP_KEY, 
    src.DUPL_RNK, 
    CURRENT_TIMESTAMP(), 
    13165092, 
    CURRENT_TIMESTAMP(), 
    13165092
);